# Basalt MonitorSDK Demo

This notebook demonstrates the asynchronous functionality of the MonitorSDK in the Basalt Python SDK.

In [ ]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))  # Needed to make notebook work in VSCode

os.environ["BASALT_BUILD"] = "development"

from basalt import Basalt
from basalt.resources.monitor.monitorsdk_types import (
    ExperimentParams,
    GenerationParams,
    LogParams,
    TraceParams,
)

# Initialize the SDK
basalt = Basalt(
    api_key="sk-f50...",  # Replace with your API key
    log_level="debug"  # Optional: Set log level
)

## Example 1: Asynchronously Create a Trace

This example demonstrates how to create a trace asynchronously.

In [ ]:
def create_trace():
    trace_params = TraceParams(
        name="Async Test Trace",
        metadata={"source": "async_example", "type": "demo"}
    )

    trace = basalt.monitor.create_trace(
        slug="async-test-trace",
        params=trace_params
    )


    return trace

# Run the async function
trace = create_trace()

## Example 2: Create a Generation

This example demonstrates how to create a generation associated with a trace.

In [ ]:
def create_generation(trace):
    gen_params = GenerationParams(
        trace_id=trace.id,
        text="This is an async test generation",
        model_id="gpt-4",
        prompt="Generate a response asynchronously",
        metadata={"source": "async_example", "type": "demo"}
    )

    generation = basalt.monitor.create_generation(gen_params)


    return generation

# Run the async function
generation = create_generation(trace)

## Example 3: Create a Log

This example demonstrates how to create a log entry associated with a trace.

In [ ]:
def create_log(trace):
    log_params = LogParams(
        trace_id=trace.id,
        type="info",
        message="This is an async test log message",
        metadata={"source": "async_example", "type": "demo"}
    )

    log = basalt.monitor.create_log(log_params)


    return log

# Run the async function
log = create_log(trace)

## Example 4: Create an Experiment

This example demonstrates how to create an experiment.

In [ ]:
async def create_experiment():
    exp_params = ExperimentParams(
        type="A/B Test",
        name="Async Test Experiment",
        setup={
            "control_id": "control-prompt",
            "variation_id": "test-prompt"
        }
    )

    err, experiment = await basalt.monitor.create_experiment(
        "async-test-feature",
        exp_params
    )


    return experiment

# Run the async function
experiment = await create_experiment()

## Example 5: Execute Multiple Monitoring Operations Concurrently

This example demonstrates how to execute multiple asynchronous monitoring operations concurrently for better performance.

In [ ]:
import asyncio


async def execute_concurrent_operations():

    # Create trace parameters for concurrent operations
    trace_params1 = TraceParams(
        name="Concurrent Trace 1",
        metadata={"source": "async_concurrent_example", "trace_number": 1}
    )

    trace_params2 = TraceParams(
        name="Concurrent Trace 2",
        metadata={"source": "async_concurrent_example", "trace_number": 2}
    )

    # Create multiple async tasks
    tasks = [
        basalt.monitor.create_trace("concurrent-trace-1", trace_params1),
        basalt.monitor.create_trace("concurrent-trace-2", trace_params2)
    ]

    # Execute all tasks concurrently
    await asyncio.gather(*tasks)


# Run the async function
await execute_concurrent_operations()